In [106]:
register_status = {}

# Registradores inteiros (x0..x31)
for i in range(32):
    register_status[f"x{i}"] = {"x": i, "x_type": "int", "writer": None}

# Registradores de ponto flutuante (f0..f31)
for i in range(32):
    register_status[f"f{i}"] = {"f": i, "f_type": "float", "writer": None}

register_status


{'x0': {'x': 0, 'x_type': 'int', 'writer': None},
 'x1': {'x': 1, 'x_type': 'int', 'writer': None},
 'x2': {'x': 2, 'x_type': 'int', 'writer': None},
 'x3': {'x': 3, 'x_type': 'int', 'writer': None},
 'x4': {'x': 4, 'x_type': 'int', 'writer': None},
 'x5': {'x': 5, 'x_type': 'int', 'writer': None},
 'x6': {'x': 6, 'x_type': 'int', 'writer': None},
 'x7': {'x': 7, 'x_type': 'int', 'writer': None},
 'x8': {'x': 8, 'x_type': 'int', 'writer': None},
 'x9': {'x': 9, 'x_type': 'int', 'writer': None},
 'x10': {'x': 10, 'x_type': 'int', 'writer': None},
 'x11': {'x': 11, 'x_type': 'int', 'writer': None},
 'x12': {'x': 12, 'x_type': 'int', 'writer': None},
 'x13': {'x': 13, 'x_type': 'int', 'writer': None},
 'x14': {'x': 14, 'x_type': 'int', 'writer': None},
 'x15': {'x': 15, 'x_type': 'int', 'writer': None},
 'x16': {'x': 16, 'x_type': 'int', 'writer': None},
 'x17': {'x': 17, 'x_type': 'int', 'writer': None},
 'x18': {'x': 18, 'x_type': 'int', 'writer': None},
 'x19': {'x': 19, 'x_type': 'int

Parser das instruções

In [107]:
import parser_inst
inst_status = []
instructions = parser_inst.parse_instructions("tests/ex.s")
for inst in instructions:
    inst_status.append({"instr": inst, "issue": None, "read": None, "exec": None, "write": None})
for i in inst_status:
    print(i)

{'instr': {'opcode': 0, 'rs1': 1, 'rs1_type': 'int', 'rs2': 0, 'rs2_type': None, 'rd': 1, 'rd_type': 'float', 'imm': 0}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 0, 'rs1': 1, 'rs1_type': 'int', 'rs2': 0, 'rs2_type': None, 'rd': 5, 'rd_type': 'float', 'imm': 0}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 5, 'rs1': 4, 'rs1_type': 'float', 'rs2': 5, 'rs2_type': 'float', 'rd': 2, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 4, 'rs1': 8, 'rs1_type': 'float', 'rs2': 9, 'rs2_type': 'float', 'rd': 4, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 2, 'rs1': 2, 'rs1_type': 'float', 'rs2': 3, 'rs2_type': 'float', 'rd': 1, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 1, 'rs1': 2, 'rs1_type': 'int', 'rs2': 4, 'rs2_type': 'float

Parser das functionals units

In [108]:
fus_configs = {}
fus = {}
with open("tests/uf_config2.in", 'r') as f:
    linhas = f.readlines()
    for linha in linhas:
        partes = linha.strip().split()
        fus_configs[partes[0]] = {partes[1]: partes[2]}

for uf, config in fus_configs.items():
    for c in config:
        qnt = int(c)
        ciclos = int(config[c])
        for i in range(qnt):
            if uf == 'int':
                opcode = 0
            elif uf == 'add':
                opcode = 1
            elif uf == 'mult':
                opcode = 2
            elif uf == 'div':
                opcode = 3

            fus[f"{uf}{i+1}"] = {
                "Opcode": opcode,
                "Busy": False,
                "Op": None,
                "Fi": None,
                "Fj": None,
                "Fk": None,
                "Qj": None,
                "Qk": None,
                "Rj": True,
                "Rk": True,
                "Cycles": ciclos,
                "CyclesLeft": 0
            }

for u in fus:
    print(u, fus[u])

int1 {'Opcode': 0, 'Busy': False, 'Op': None, 'Fi': None, 'Fj': None, 'Fk': None, 'Qj': None, 'Qk': None, 'Rj': True, 'Rk': True, 'Cycles': 1, 'CyclesLeft': 0}
int2 {'Opcode': 0, 'Busy': False, 'Op': None, 'Fi': None, 'Fj': None, 'Fk': None, 'Qj': None, 'Qk': None, 'Rj': True, 'Rk': True, 'Cycles': 1, 'CyclesLeft': 0}
div1 {'Opcode': 3, 'Busy': False, 'Op': None, 'Fi': None, 'Fj': None, 'Fk': None, 'Qj': None, 'Qk': None, 'Rj': True, 'Rk': True, 'Cycles': 10, 'CyclesLeft': 0}


## ISSUE

Issue:
Pega a instrução da tabela "Instruction status" 
Verifica qual fu ela quer usar
Verifica se tem fu correspondente livre na tabela "Functional unit status"
Verifica se alguém vai escrever no registrador rd (destino) na tabela "Register Status" 
Caso não, reserva FU na fus_status_table
Reserva RD register_status


In [109]:
instr = inst_status[0]['instr']
opcode = instr['opcode']
rs1 = instr['rs1']
rs1_type = instr['rs1_type']
rs2 = instr['rs2']
rs2_type = instr['rs2_type']
rd = instr['rd']
rd_type = instr['rd_type']
# Reservar registrador de destino (rd)    
prefix_rd = "x" if rd_type == "int" else "f"
key_rd = f"{prefix_rd}{rd}"

prefix_rs1 = "x" if rs1_type == "int" else "f"
key_rs1 = f"{prefix_rs1}{rs1}"

prefix_rs2 = "x" if rs2_type == "int" else "f"
key_rs2 = f"{prefix_rs2}{rs2}"
if opcode == 0: # qual fu precisa?
    for fu_name, fu in fus.items(): 
        if fu['Opcode'] == opcode and not fu['Busy']: # se a FU precisa estiver livre
            fu['Busy'] = True # Reserva a FU
            fu['Op'] = opcode
            fu['Fi'] = rd
            fu['Fj'] = rs1
            fu['Fk'] = rs2
            fu['Qj'] = register_status[key_rs1]['writer']
            fu['Qk'] = register_status[key_rs2]['writer']
            fu['Rj'] = True if fu['Qj'] is None else False
            fu['Rk'] = True if fu['Qk'] is None else False
            fu['CyclesLeft'] = fu['Cycles']
            inst_status[0]['issue'] = 1 # Ciclo de issue
            


In [110]:
for i in inst_status:
    print(i)

{'instr': {'opcode': 0, 'rs1': 1, 'rs1_type': 'int', 'rs2': 0, 'rs2_type': None, 'rd': 1, 'rd_type': 'float', 'imm': 0}, 'issue': 1, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 0, 'rs1': 1, 'rs1_type': 'int', 'rs2': 0, 'rs2_type': None, 'rd': 5, 'rd_type': 'float', 'imm': 0}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 5, 'rs1': 4, 'rs1_type': 'float', 'rs2': 5, 'rs2_type': 'float', 'rd': 2, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 4, 'rs1': 8, 'rs1_type': 'float', 'rs2': 9, 'rs2_type': 'float', 'rd': 4, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 2, 'rs1': 2, 'rs1_type': 'float', 'rs2': 3, 'rs2_type': 'float', 'rd': 1, 'rd_type': 'float', 'imm': None}, 'issue': None, 'read': None, 'exec': None, 'write': None}
{'instr': {'opcode': 1, 'rs1': 2, 'rs1_type': 'int', 'rs2': 4, 'rs2_type': 'float', 

## Scoreboard

In [113]:
# instr = inst_status[0]['instr']

for instr in inst_status:
    print(instr['instr'])    

{'opcode': 0, 'rs1': 1, 'rs1_type': 'int', 'rs2': 0, 'rs2_type': None, 'rd': 1, 'rd_type': 'float', 'imm': 0}
{'opcode': 0, 'rs1': 1, 'rs1_type': 'int', 'rs2': 0, 'rs2_type': None, 'rd': 5, 'rd_type': 'float', 'imm': 0}
{'opcode': 5, 'rs1': 4, 'rs1_type': 'float', 'rs2': 5, 'rs2_type': 'float', 'rd': 2, 'rd_type': 'float', 'imm': None}
{'opcode': 4, 'rs1': 8, 'rs1_type': 'float', 'rs2': 9, 'rs2_type': 'float', 'rd': 4, 'rd_type': 'float', 'imm': None}
{'opcode': 2, 'rs1': 2, 'rs1_type': 'float', 'rs2': 3, 'rs2_type': 'float', 'rd': 1, 'rd_type': 'float', 'imm': None}
{'opcode': 1, 'rs1': 2, 'rs1_type': 'int', 'rs2': 4, 'rs2_type': 'float', 'rd': 0, 'rd_type': None, 'imm': 0}
